In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
train_dir = 'train'

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
W = tf.Variable(tf.zeros([784, 10]), name='W')
b = tf.Variable(tf.zeros([10]), name='b')
y = tf.nn.softmax(tf.matmul(x, W) + b, name='y')

In [ ]:
yt = tf.placeholder(tf.float32, [None, 10], name='yt')
cross_entropy = tf.reduce_mean(-tf.reduce_sum(yt * tf.log(y), reduction_indices=[1]), name='cross_entropy')
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(yt,1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

In [ ]:
saver = tf.train.Saver(tf.trainable_variables())
sess = tf.InteractiveSession()

tf.add_to_collection('x', x)
tf.add_to_collection('yt', yt)
tf.add_to_collection('accuracy', accuracy)

In [ ]:
tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, yt: batch_ys})
    if i % 50 == 49:
        os.makedirs(train_dir, exist_ok=True)
        checkpoint_path = '%s/mnist.ckpt' % train_dir
        saver.save(sess, checkpoint_path, global_step=i)

In [4]:
with tf.Session() as sess:
    train_dir = 'train'
    latest_checkpoint = tf.train.latest_checkpoint(train_dir)
    meta_path = '%s.meta' % latest_checkpoint

    saver = tf.train.import_meta_graph(meta_path)
    saver.restore(sess, latest_checkpoint)
    
    x = tf.get_collection('x')[0]
    yt = tf.get_collection('yt')[0]
    accuracy = tf.get_collection('accuracy')[0]

    feed_dict={x: mnist.test.images, yt: mnist.test.labels}

    print(sess.run(accuracy, feed_dict))

INFO:tensorflow:Restoring parameters from train/mnist.ckpt-999
0.9109


In [ ]:
num_rows, num_cols = 10, 10
width, height = num_cols * 2, num_rows * 2
fig = plt.figure(figsize=(width, height))

for i in range(0, 100):
    ax = fig.add_subplot(num_rows, num_cols, i+1)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    image = mnist.test.images[i].reshape((28, 28))
    label = mnist.test.labels[i].argmax()
    prediction = sess.run(y, feed_dict={x: mnist.test.images[i:i+1]}).argmax()
    ax.imshow(image, cmap='Greys', interpolation='none')
    text = u'%i=%i'%(label, prediction) if label==prediction else u'%i≠%i'%(label, prediction)
    color = 'black' if label==prediction else 'red'
    ax.text(
        0, 
        0, 
        text, 
        bbox={'facecolor':'white', 'pad':5}, 
        fontdict={'size':14, 'weight': 'bold', 'color': color}
    )